# Kalman filter

## 理論概要

<b>補正後の状態　＝　補正前の状態 ＋ カルマンゲイン ×（本物の観測値－予測された観測値）</b><br><br>
<b>カルマンゲイン ＝ 状態の予測誤差の分散　÷（状態の予測誤差の分散＋観測方程式のノイズの分散）</b>
<br><br>
将来予測が当たる（「状態の予測誤差の分散」が小さい）と判断されていれば、補正は少なくなります。<br>
また、観測方程式におけるノイズが大きいと判断されても、やはり補正幅は小さくなります。<br><br>
<b>今期の状態の予測誤差の分散 ＝ 前期の状態の予測誤差の分散 ＋ 状態方程式のノイズの分散</b>
<br><br>
ただし、これもやはり「本物の観測値」が手に入ると、予測誤差の分散も小さくなります。<br>
「観測値」を基にして「状態の予測誤差の分散」を修正するのも、カルマンフィルタの役割です。<br><br>
<b>補正後の状態の予測誤差の分散 ＝（１ーカルマンゲイン）× 補正前の状態の予測誤差の分散</b>


## ローカルレベルモデルでの実装
<br>
観測方程式<br>
「観測値」　～　「状態」 ＋　ノイズ
<br><br>
状態方程式<br>
「状態」　～　「前期の状態」　＋　ノイズ<br><br>
y : 当期の観測値<br>
x_pre : 前期の状態<br>
p_pre : 前期の状態の予測誤差の分散<br>
sigmaW : 状態方程式のノイズの分散<br>
sigmaV : 観測方程式のノイズの分散<br>
<br>

In [36]:
import pandas as pd
import numpy as np

In [35]:
def local_level_model(y,x_pre,p_pre,sigmaW,sigmaV):
    x_frc = x_pre # ローカルレベルモデルゆえ予測値は前期の値と同じ
    p_frc = p_pre + sigmaW #状態の予測誤差の分散
    k_gain = p_frc / (p_frc + sigmaW) #カルマンゲイン
    x_flt = x_frc + k_gain * (y - x_frc) #カルマンゲインを使って補正された状態
    p_flt = (1 - k_gain) * p_frc # 補正された状態の予測誤差の分散
    return x_flt,p_flt

In [32]:
df = pd.read_csv('../TagGame/Tag_simulation/oni.csv')
df.head(3)

,x,y
0,40.0580,-41.4635
1,37.1892,-42.8135
2,34.6861,-44.1072


In [2]:
#サンプルサイズ
N = len(df.x)
#状態の推定値
x = pd.Series(np.zeros(N))
y = pd.Series(np.zeros(N))
#状態の予測誤差の分散
px = pd.Series(np.zeros(N))
py = pd.Series(np.zeros(N))

#状態の初期値0
x[0] = 0
y[0] = 0
#状態の分散1000
px[0] = 1000
py[0] = 1000

print(x)
print(py)

NameError: name 'df' is not defined

In [54]:
for i in range(0,N):
    x[i+1],px[i+1] = local_level_model(df.x[i],x[i],px[i],sigmaW=1000,sigmaV=10000)
    y[i+1],py[i+1] = local_level_model(df.y[i],y[i],py[i],sigmaW=1000,sigmaV=10000)

predict = pd.DataFrame({
    'x' : x,
    'y' : y
})
predict.to_csv('../TagGame/Tag_simulation/oni_predict.csv')